# PERHATIAN!

Soal yang Anda kerjakan akan dinilai secara otomatis oleh sistem dari Dicoding. Oleh karena itu, Anda **tidak diperbolehkan mengubah nama fungsi yang sudah ditentukan** karena hal tersebut dapat memengaruhi proses penilaian.

Pastikan Anda membaca dan memperhatikan setiap instruksi dengan saksama,serta menuliskan kode di antara tanda komentar yang telah disediakan.

> **# MULAI KODE DI SINI**


> **# AKHIRI KODE DI SINI**

In [33]:
import math
import numpy as np
import pandas as pd
import random
import string

import scipy.stats as stats
from dataclasses import dataclass

# Latar Belakang Masalah

Anda telah mempelajari A/B testing pada modul khusus di kelas Belajar Matematika untuk Data Science.

Pada soal ini, Anda akan mengimplementasikan uji hipotesis melalui sebuah studi kasus A/B testing yang ada di industri.

---
Anda adalah seorang data scientist di perusahaan teknologi pendidikan yang telah beroperasi selama bertahun-tahun. Saat ini, perusahaan sedang mengembangkan fitur baru untuk diterapkan pada aplikasi Anda.

Salah satu indikator keberhasilan fitur adalah kemampuannya dalam meningkatkan conversion rate, yaitu persentase pengguna yang melakukan tindakan yang diinginkan dibandingkan dengan total pengguna.

Fitur-fitur yang dikembangkan kali ini bertujuan meningkatkan retensi pengguna. Saat ini, rata-rata pengguna bertahan di situs sebesar 69% dan diharapkan meningkat menjadi 72% dengan adanya fitur ini.

# Tugas 1: Menghitung Proporsi Sampel yang Dibutuhkan

Dalam melakukan pengujian hipotesis, proporsi sampel yang akan digunakan harus cukup besar dan kuat untuk bisa menghasilkan uji hipotesis yang signifikan.

Saat ini, Anda ingin meningkatkan rasio retensi pengguna dari yang awalnya 69% menjadi 72%. Kira-kira berapa sampel yang dibutuhkan? Simak penjelasannya di bawah ini.

---

Untuk membandingkan dua proporsi dengan studi kasus yang telah disebutkan. Anda akan melakukan uji hipotesis menggunakan uji dua sisi (two-tailed z-test).

Adapun hipotesis yang diuji sebagai berikut.
* Hipotesis nol ($H_0$): Tidak ada perbedaan antara kedua proporsi $(p_1 = p2)$.
* Hipotesis alternatif ($H_1$): Ada perbedaan antara kedua proporsi $(p_1 \neq p2)$.

Dengan asumsi bahwa ukuran kelompok kedua adalah $k$ kali lebih besar dari kelompok pertama, kita bisa menggunakan rumus berikut untuk menghitung **ukuran sampel minimal yang dibutuhkan**.

$$n_1 = \frac{\left[\sqrt{\bar{p}\bar{q}\left(1 + \frac{1}{k} \right)}z_{1- \alpha/2} + \sqrt{p_1 q_1 + \frac{p_2q_2}{k}}z_{1-\beta}\right]^2}{\Delta^2}$$


$$n_2 = k n_1$$

Artinya:
* $p_1,p_2$ merupakan proporsi keberhasilan yang diasumsikan untuk masing-masing kelompok.
* $q_1 = 1 - p_1$ dan $q_2 = 1 - p_2$ adalah proporsi kegagalan.
* $\Delta  = \mid p_2 - p_1 \mid$ merupakan selisih absolut antara dua proporsi.
* $\overline{p} = \frac{p_1 + kp_2}{1+ k}$ adalah rata-rata dari kedua proporsi dengan k sebagai rasio ukuran antara kelompok kedua dan pertama.

In [25]:
def estimate_sample_size_proportions(p1, p2, alpha=0.05, beta=0.20, two_sided=True):
    """
    Menghitung jumlah sampel minimum yang dibutuhkan untuk mengetahui perbedaan dua proporsi (Uji dua proporsi).

    Params:
    - p1 (float): Proporsi keberhasilan pada kelompok origin.
    - p2 (float): Proporsi keberhasilan pada kelompok varian.
    - alpha (float): Tingkat Signifikansi (default 0.05).
    - beta (float): Probabilitas kesalahan tipe II (default 0.20, artinya power 80%).
    - two_sided (bool): Jika True, gunakan uji dua sisi.

    Return:
    - n (int): Jumlah sampel minimum per kelompok.
    """

    ## JANGAN UBAH KODE BAGIAN INI ##
    k = 1  # Mengasumsikan kelompok sama besar.

    # Menghitung peluang kegagalan dari masing-masing kelompok
    q1 = (1 - p1)
    q2 = (1 - p2)

    # Rata-rata dari dua proporsi
    p_bar = (p1 + k * p2) / (1 + k)
    q_bar = 1 - p_bar

    # Selisih absolut antara dua proporsi
    delta = abs(p2 - p1)

    # Jika uji dua sisi, sesuaikan nilai alpha
    if two_sided:
          alpha = alpha / 2
    ## HINGGA BAGIAN KODE INI

    # Mendefinisikan z-score dari distribusi normal untuk alpha dan beta.
    z_alpha = stats.norm.ppf(1 - (alpha)) # z1−α/2
    z_beta = stats.norm.ppf(1 - (beta))   # z1−β

    # MULAI KODE DI SINI
    # Menghitung jumlah sampel minimum.
    # Petunjuk: Gunakan rumus n1 yang sudah disebutkan sebelumnya.
    n = ((z_alpha * math.sqrt((1 + k) * p_bar * q_bar) + z_beta * math.sqrt(k * p1 * q1 + p2 * q2)) ** 2) / (delta ** 2)
    
    # AKHIRI KODE DI SINI

    return math.ceil(n)

In [26]:
# Pengujian: Silakan gunakan cell ini untuk menguji fungsi yang Anda buat. Pastikan output-nya sesuai dengan yang diharapkan.

sample_size_needed = estimate_sample_size_proportions(0.69, 0.72)
sample_size_needed

3627

Output yang diharapkan

```
3627
```

Keren! Kita telah mengetahui bahwa dibutuhkan 3.627 pengguna per kelompok untuk dapat mendeteksi perbedaan yang signifikan antara dua kelompok dengan tingkat signifikansi 0,05 dan power level 0,8.

Permasalahannya, jumlah pengguna aktif harian kita hanya sekitar 997 orang. Lalu, berapa lama A/B test perlu dijalankan?

Untuk menghitungnya, kita dapat membagi total sample size needed dengan jumlah pengguna aktif per hari. Karena pada pengujian ini kita membagi pengguna secara merata (50:50) dalam dua grup, rumus yang digunakan adalah sebagai berikut.

$$
\frac{2 \cdot \text{sample_size_needed}}{\text{active_users}}
$$


In [27]:
active_users_per_day = 997

# MULAI KODE DI SINI
# TODO: Hitung banyaknya hari yang dibutuhkan dengan rumus di atas.
n_days = math.ceil(sample_size_needed / active_users_per_day)
# AKHIRI KODE DI SINI

print(f"AB tes harus dijalankan selama {n_days} hari untuk mendapatkan data yang cukup.")

AB tes harus dijalankan selama 4 hari untuk mendapatkan data yang cukup.


# Tugas 2: Membuat Data Pengguna

Sampai tahap ini, kita sudah mengetahui bahwa pengujian harus dijalankan selama 8 hari karena asumsinya setiap hari pengguna aktif berkisar pada angka 997.

Dengan begitu, mari buat dataset yang menyimulasikan pengguna yang mengunjungi aplikasi Anda. Data tersebut akan terdiri dari tiga kolom utama berikut.
* **user\_id**: ID atau identitas unik dari masing-masing pengguna.
* **user\_type**: Menyatakan tipe pengguna, yaitu `origin` atau `varian`.
* **converted**: Menunjukkan jumlah artikel yang dibaca oleh pengguna.

In [28]:
# JANGAN MENGUBAH FUNGSI INI #
def create_unique_user_ids(num_users):
  """
  Menghasilkan sejumlah ID pengguna acak yang terdiri dari kombinasi huruf kapital ASCII dan angka
  dengan panjang tetap sebanyak 10 karakter.

  Parameter:
  - num_users (int): Jumlah ID unik yang ingin dihasilkan.

  Return:
  - user_ids (List[str]): List berisi ID unik hasil generate.
  """
  user_ids = []

  # Pengulangan hingga jumlah user_id yang dihasilkan sesuai dengan yang diminta
  while len(user_ids) < num_users:

      # Menghasilkan nilai random yang merupakan acak dari ascii dan string digits.
      new_id = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))

      if new_id not in user_ids:
        user_ids.append(new_id)

  return list(user_ids)

In [29]:
def generate_df_ab_test(n_days):

    ## JANGAN UBAH BAGIAN KODE DARI SINI
    np.random.seed(69)

    daily_users = 499
    n_origin = int(daily_users*n_days*np.random.uniform(0.98, 1.02))
    n_varian = int(daily_users*n_days*np.random.uniform(0.98, 1.02))

    data_origin = np.random.choice([0, 1], size=n_origin, p=[1-0.69, 0.69])
    data_varian = np.random.choice([0, 1], size=n_varian, p=[1-0.73, 0.73])
    ## HINGGA BAGIAN KODE INI

    ## MULAI KODE DI SINI
    
    # Buat user ID dengan menggunakan fungsi create_unique_user_ids yang telah
    # didefinisikan sebelumnya. Gunakan argumen berupa total jumlah pengguna: n_origin + n_varian.
    user_ids = create_unique_user_ids(n_origin + n_varian)
    origin_user_ids = user_ids[:n_origin]
    varian_user_ids = user_ids[n_origin:]

    # Buat DataFrame untuk masing-masing kelompok: origin dan varian dengan ketentuan sebagai berikut.
    # - Kolom: user_id, user_type, dan converted
    # - Kolom user_type diisi dengan "origin" untuk kelompok origin dan "varian" untuk kelompok varian.
    # - Kolom converted untuk kelompok origin, diisi dengan variabel data_origin, begitu pun untuk varian.
    origin_dict = {
        'user_id': origin_user_ids,
        'user_type': ['origin'] * n_origin,
        'converted': data_origin
    }
    varian_dict = {
        'user_id': varian_user_ids,
        'user_type': ['varian'] * n_varian,
        'converted': data_varian
    }

    origin_df = pd.DataFrame(origin_dict)
    varian_df = pd.DataFrame(varian_dict)
    ## AKHIRI KODE DI SINI

    df_ab_test = pd.concat([origin_df, varian_df]).sample(frac=1).reset_index(drop=True)

    return df_ab_test

Pada tahap ini, data telah berhasil dibuat. Untuk melihatnya lebih detail, mari lihat rata-rata `convertion_rate` pada masing-masing kelompok.

In [34]:
# Pengujian: Silakan gunakan cell ini untuk menguji fungsi yang Anda buat. Pastikan output-nya sesuai dengan yang diharapkan.

df = generate_df_ab_test(n_days)

origin_data = df[df["user_type"]=="origin"]["converted"]
varian_data = df[df["user_type"]=="varian"]["converted"]

print(f"{len(origin_data)} pengguna melihat website asli dengan tingkat rasio konversi (converstion_rate) adalah {origin_data.mean():.4f}")
print(f"{len(varian_data)} pengguna melihat website dengan fitur baru memiliki tingkat rasio konversi (converstion_rate) adalah {varian_data.mean():.4f}")

1979 pengguna melihat website asli dengan tingkat rasio konversi (converstion_rate) adalah 0.6842
2020 pengguna melihat website dengan fitur baru memiliki tingkat rasio konversi (converstion_rate) adalah 0.7401


```
3959 pengguna melihat website asli dengan tingkat rasio konversi (converstion_rate) adalah 0.6906
4041 pengguna melihat website dengan fitur baru memiliki tingkat rasio konversi (converstion_rate) adalah 0.7191
```

Sekilas, kita dapat melihat bahwa pembagian pengguna antar kelompok dilakukan secara merata. Selain itu, kelompok varian (yaitu pengguna yang mencoba fitur baru) menunjukkan rasio konversi yang lebih tinggi dibandingkan kelompok kontrol.

Temuan ini mengindikasikan bahwa pengguna dalam kelompok varian kemungkinan lebih berhasil engage dengan website dibandingkan mereka yang menggunakan versi tanpa fitur baru. Namun demikian, kita tahu bahwa rata-rata saja belum cukup untuk menarik kesimpulan yang valid secara statistik.

Untuk memastikan bahwa perbedaan tersebut benar-benar signifikan, Anda perlu melakukan pengujian hipotesis. Karena yang dibandingkan adalah proporsi keberhasilan antar dua kelompok, Anda dapat menggunakan z-test untuk proporsi sebagai metode pengujian.

Berikut adalah rumus yang akan digunakan (Ini adalah rumus z test untuk dua proporsi).
$$ z = \frac{\hat{p}_1 - \hat{p}_2}{\sqrt{\hat{p}(1-\hat{p})\left(\frac{1}{n_1} + \frac{1}{n_2}\right)}}$$

Dengan $\hat{p}$ adalah *proporsi gabungan*: $\hat{p} = \frac{x_1 + x_2}{n_1 + n_2}$

Pada tugas-tugas selanjutnya, kita akan berfokus pada rumus-rumus tersebut. Mari kita mulai dengan menyiapkan metrik untuk rumus tersebut.

---
Di bawah ini, kita akan menyimpan nilai-nilai variabel x, n, dan p dalam `dataclass`.
Jangan khawatir jika Anda belum familier dengan fungsi ini. dataclass adalah decorator pada Python (dikenalkan sejak Python 3.7) yang secara otomatis menambahkan *boilerplate code* dalam sebuah class untuk menyimpan data.

Contohnya, Anda memiliki class berikut.

```
@dataclass
class my_course:
  course_id: int
  course_name: string

foo = my_course(course_name='Dicoding')
```

Anda dapat mengakses informasi `course_name` dari `foo` melalui sintaks `foo.course_name` yang akan mengembalikan string "Dicoding".


> Mengapa menggunakan dataclass? Anda perlu menyimpan parameter distribusi untuk setiap fitur. Misalnya, untuk weight dan height, diperlukan parameter μ dan σ. Jika dibuat terpisah, seperti miu_weight, sigma_weight, miu_height, sigma_height, akan tidak efektif karena terlalu panjang dan beragam.

__repr__ adalah method yang digunakan untuk menampilkan objek saat Anda print.

Tanpa `__repr__`:
```
<my_class object at 0x7f9f3b4d>
```

Dengan `__repr__`:
```
my_class(a='Dicoding')
```

In [35]:
@dataclass
class metrics_estimation:
    n: int
    x: int
    p: float

    def __repr__(self):
        return f"sample_params(n={self.n}, x={self.x}, p={self.p:.3f})"

# Tugas 3: Menghitung Metrik untuk Z-Test

Setelah variabel disiapkan, silakan lengkapi fungsi di bawah ini untuk menghitung nilai z-test berdasarkan rumus yang disebutkan sebelumnya.

Anda hanya perlu menghitung untuk masing-masing nilai berikut.

*   n: Banyaknya pengguna dalam data.
*   x: Banyaknya pengguna yang berhasil dikonversi dalam data.
*   p: Rasio konversi (conversion rate).


<details>
<summary>
<font color='yellow'>PETUNJUK!</font>
</summary>
- Fungsi ini menerima data dalam bentuk Pandas Series. Anda dapat memanfaatkan fungsi-fungsi, seperti .sum() untuk jumlah atau /mean() untuk rata-rata.

In [37]:
def generate_proportion_metrics(data):
  """
  Menghitung metrik relevan dari sampel untuk data, seperti proporsi.

  Parameters:
  - data (pandas.core.series.Series): Data Sampel.

  Returns:
  - metrics_estimation: Metrik yang disimpan sebagai objek class `estimation_metrics`.
  """


  ## MULAI KODE DI SINI

  # Hitung banyaknya data dalam sampel (n)
  # Hitung total keberhasilan (jumlah data bernilai 1)
  # Hitung proporsi keberhasilan dari seluruh sampel (p)
  # Simpan semua nilai tersebut dalam objek metrics_estimation (nama class yang sudah didefinisikan sebelumnya)
  metrics = metrics_estimation(
      n=len(data),
      x=int(data.sum()),
      p=float(data.mean())
  )
  ## AKHIRI KODE DI SINI

  return metrics

In [38]:
# Pengujian: Silakan gunakan cell ini untuk menguji fungsi yang Anda buat. Pastikan output-nya sesuai dengan yang diharapkan.

origin_metrics = generate_proportion_metrics(origin_data)
print(f"Kelompok origin: n={origin_metrics.n}, x={origin_metrics.x}, dan p={origin_metrics.p:.4f}\n")

varian_metrics = generate_proportion_metrics(varian_data)
print(f"Kelompok varian: n={varian_metrics.n}, x={varian_metrics.x}, dan p={varian_metrics.p:.4f}")

Kelompok origin: n=1979, x=1354, dan p=0.6842

Kelompok varian: n=2020, x=1495, dan p=0.7401


Output yang diharapkan
```
Kelompok origin: n=3959, x=2734, dan p=0.6906

Kelompok varian: n=4041, x=2906, dan p=0.7191
```

# Tugas 4: Menghitung Z-Test (Bagian: Proporsi Gabungan)

Sampai tahap ini, Anda sudah menyiapkan setiap variabel yang akan digunakan dalam rumus. Mari ingat kembali rumus yang akan digunakan.

$$ z = \frac{\hat{p}_1 - \hat{p}_2}{\sqrt{\hat{p}(1-\hat{p})\left(\frac{1}{n_1} + \frac{1}{n_2}\right)}}$$

$\hat{p}$ adalah *proporsi gabungan*.

Untuk melakukan kalkulasi terhadap z-test, kita perlu melakukan perhitungan proporsi gabungan dengan rumus berikut.
$$\hat{p} = \frac{x_1 + x_2}{n_1 + n_2}$$

Mari lengkapi fungsi di bawah ini untuk menghitung proporsi gabungan berdasarkan rumus di atas.

In [39]:
def compute_pooled_proportion(origin_metrics, varian_metrics):
  """
  Menghitung proporsi gabungan dari dua sampel untuk analisis perbandingan proporsi.

  Parameters:
  - origin_metrics (metrics_estimation): Objek yang berisi metrik untuk z-test dari kelompok origin.
  - varian_metrics (metrics_estimation): Objek yang berisi metrik untuk z-test dari kelompok varian.

  Returns:
  - pooled_proportions (numpy.float): Proporsi Gabungan. Anda menggunakan rumus z-test untuk dua proporsi untuk menyelesaikan studi kasus yang dihadapi. Dengan rumusnya di bawah ini.


Nilai p^ adalah proporsi gabungan, apakah rumus untuk menghitung proporsi gabungan?

  """

  ## MULAI KODE DI SINI
  # Ambil variabel x dan n dari masing-masing kelompok.
  x1 = origin_metrics.x
  n1 = origin_metrics.n
  x2 = varian_metrics.x
  n2 = varian_metrics.n
  # Hitung proporsi gabungan dengan rumus yang disebutkan untuk proporsi gabungan.
  pooled_proportions = (x1 + x2) / (n1 + n2)
  ## AKHIRI KODE DI SINI

  return pooled_proportions

In [40]:
# Pengujian: Silakan gunakan cell ini untuk menguji fungsi yang Anda buat. Pastikan output-nya sesuai dengan yang diharapkan.

pp = compute_pooled_proportion(origin_metrics, varian_metrics)
print(f"Proporsi gabungan untuk data sampel AB Test: {pp:.4f}")

Proporsi gabungan untuk data sampel AB Test: 0.7124


Output yang diharapkan
```
Proporsi gabungan untuk data sampel AB Test: 0.7050
```

# Tugas 5: Menghitung Z-Test (Bagian: Rumus Utama)

Nilai untuk proporsi gabungan telah rampung dibuat! Tentunya, langkah berikutnya adalah menggunakan fungsi tersebut untuk rumus z-test yang disebutkan sebelumnya.

In [41]:
def z_statistic_diff_proportions(origin_metrics, varian_metrics):
  """
  Menghitung z-statistic untuk perbedaan kedua proporsi.

  Parameters:
  - origin_metrics (metrics_estimation): Metrik untuk sampel kelompok origin.
  - varian_metrics (metrics_estimation): Metrik untuk sampel kelompok varian.

  Returns:
  - z (numpy.float): Nilai z-statistic
  """

  ## MULAI KODE DI SINI
  # Petunjuk:
  # - Gunakan fungsi compute_pooled_propotion untuk nilai p_hat pada rumus z-statistic.
  # - Manfaatkan library numpy untuk melakukan kalkulasi akar kuadrat.
  p_hat = compute_pooled_proportion(origin_metrics, varian_metrics)
  p1 = origin_metrics.p
  p2 = varian_metrics.p
  q_hat = 1 - p_hat
  z = (p2 - p1) / np.sqrt(p_hat * q_hat * (1/origin_metrics.n + 1/varian_metrics.n))
  ## AKHIRI KODE DI SINI

  return z

In [42]:
# Pengujian: Silakan gunakan cell ini untuk menguji fungsi yang Anda buat. Pastikan output-nya sesuai dengan yang diharapkan.

z = z_statistic_diff_proportions(origin_metrics, varian_metrics)
print(f"Nilai z statistic untuk AB test adalah {z:.4f}")

Nilai z statistic untuk AB test adalah 3.9058


Output yang diharapkan
```
Nilai z statistic untuk AB test adalah -2.7996
```

# Tugas 6: Menolak/Menerima Hipotesis

Setelah melakukan semua perhitungan z-statistic, sekarang saatnya menentukan jika hipotesis nol (null hypothesis) akan ditolak atau tidak.

Untuk dapat menentukan hal tersebut, Anda perlu mencari nilai p-value berdasarkan nilai z-statistic dan tingkat signifikansi (alpha) yang ditentukan.

Dalam konteks ini, p-value merepresentasikan probabilitas untuk memperoleh nilai z-statistic yang sama ekstremnya atau lebih ekstrem dibanding nilai yang diamati, jika hipotesis nol benar.

Anda dapat menggunakan fungsi CDF (cumulative distribution function) dari distribusi normal untuk menghitung probabilitas bahwa nilai yang diperoleh kurang dari atau sama dengan nilai yang diamati.

Oleh karena itu, p-value dihitung menggunakan rumus berikut.

$$
p = 2 \times (1 - \text{CDF}(|z|))
$$

<details> <summary><font color='yellow'>PETUNJUK</font></summary>

* Anda dapat menggunakan fungsi stats.norm.cdf dari scipy.stats untuk menghitung p-value.

* Gunakan nilai absolut dari z-statistic karena melakukan uji dua sisi (two-sided test) sehingga kita memperhitungkan kedua ekor distribusi.

* Kalikan hasilnya dengan 2 untuk mencerminkan kedua sisi.

* Jika p-value lebih kecil dari alpha, hipotesis nol ditolak.

</details>

In [43]:
def reject_nh_z_statistic(z_statistic, alpha=0.05):
  """
  Menentukan diterima atau ditolaknya hipotesis nol dalam z-test.

  Parameters:
  - z_statistics (float): Nilai z-statistik dari dua proporsi.
  - alpha (float, opsional): Tingkat signifikansi yang digunakan. Nilai default adalah 0.05

  Returns:
  - reject (bool): Bernilai True jika hipotesis nol harus ditolak dan False jika tidak ditolak.

  """

  reject = False

  ## MULAI KODE DI SINI
  # Hitung nilai p_value untuk uji dua sisi berdasarkan rumus yang dijelaskan sebelumnya.
  p_value = 2 * (1 - stats.norm.cdf(abs(z_statistic)))

  # Bandingkan nilai p_value dengan alpha. Jika p_value lebih kecil, tolak hipotesis nol.
  if p_value < alpha:
      reject = True
  ## AKHIRI KODE DI SINI

  return reject

In [44]:
# Pengujian: Silakan gunakan cell ini untuk menguji fungsi yang Anda buat. Pastikan output-nya sesuai dengan yang diharapkan.

alpha = 0.05
tolak_nh = reject_nh_z_statistic(z, alpha)

print(f"Hipotesis nol {'' if tolak_nh else 'tidak '}dapat ditolak pada tingkat signifikansi {alpha:.2f}.\n")

pesan = "" if tolak_nh else "Tidak "
print(f"{pesan}Ada cukup bukti statistik untuk menolak H0.\n"
      f"Dengan kata lain, dapat disimpulkan bahwa ada perbedaan yang {pesan}signifikan secara statistik antara kedua proporsi.")

Hipotesis nol dapat ditolak pada tingkat signifikansi 0.05.

Ada cukup bukti statistik untuk menolak H0.
Dengan kata lain, dapat disimpulkan bahwa ada perbedaan yang signifikan secara statistik antara kedua proporsi.


Output yang diharapkan
```
Hipotesis nol dapat ditolak pada tingkat signifikansi 0.05.

Ada cukup bukti statistik untuk menolak H0.
Dengan kata lain, dapat disimpulkan bahwa ada perbedaan yang signifikan secara statistik antara kedua proporsi.
```

Hebat! Anda berhasil membuktikan bahwa ada perbedaan yang signifikan secara statistik antara kedua proporsi.

Berdasarkan temuan ini, Anda dapat menyimpulkan bahwa fitur baru yang dikembangkan berhasil meningkatkan conversion rate pengguna. Oleh karena itu, Anda dapat merekomendasikan kepada tim pengembang perangkat lunak untuk menerapkan fitur tersebut ke seluruh pengguna aplikasi.